In [17]:
import os
import numpy as np

Read books

In [18]:
# def remove_unnecessary_chars(book):
#     def isValid(c):
#         return c in "abcdefghijklmnopqrstuvwxyz1234567890 .;:?!&')(-*"
#     return ''.join(c for c in book if isValid(c))


# #file path for where the book txts are
# book_path = 'project_books/'
# book_txt_paths = []
# books = []
# books_combined = ''

# for root, dirs, files in os.walk(book_path):
#     #file names in files path
#     for file in files:
#         #create complete path to book
#         book_file_path = book_path+file
        
#         book_txt_paths.append(book_file_path )
# #         print(book_file_path)
#         #open txt file
#         try:
#             book = open(book_file_path).read()
#             #clean the text
#             book = book.replace('\n', '')
#             book = book.replace('\r', '')
#             book = book.replace('\ufeff', '')
#             book = book.replace('\xa0', '')
#             book = book.lower()
#             #have the minimum characters in the book
#             book = remove_unnecessary_chars(book)
            
#             books.append(book)
#             #combine books together
#             books_combined += book
#         #some files don't have book texts
#         except:
#             print('passed: ', book_file_path)
#             pass
        



passed:  project_books/.DS_Store


In [19]:
#store all the sentences in a txt file for quicker access
# f = open('books_combined.txt', 'w+')
# f.write(books_combined)
# f.close()

In [22]:
import os
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
books_combined = open('books_combined.txt').read()

In [6]:
print(len(books_combined))

72569456


In [7]:
#work with a subset of the books
sub_books_combined = books_combined[:5000000]

In [8]:
# len(set(books_combined))

In [9]:
char_to_int = {}
int_to_char = {}
#turn characters into integers and integers into characters
for i, char in enumerate(sorted(set(sub_books_combined))):
    char_to_int[char] = i
    int_to_char[i] = char

In [10]:
print(len(char_to_int))
char_to_int

48


{'4': 0,
 '8': 1,
 'p': 2,
 'k': 3,
 'l': 4,
 'm': 5,
 'i': 6,
 '9': 7,
 'v': 8,
 'n': 9,
 'z': 10,
 '!': 11,
 'u': 12,
 'g': 13,
 '0': 14,
 '5': 15,
 'q': 16,
 ';': 17,
 'j': 18,
 'w': 19,
 '6': 20,
 ':': 21,
 '.': 22,
 '-': 23,
 '*': 24,
 '(': 25,
 'y': 26,
 'e': 27,
 'x': 28,
 'b': 29,
 'c': 30,
 ' ': 31,
 '1': 32,
 'f': 33,
 't': 34,
 '3': 35,
 'h': 36,
 '7': 37,
 '&': 38,
 's': 39,
 '2': 40,
 'o': 41,
 'a': 42,
 ')': 43,
 'r': 44,
 'd': 45,
 "'": 46,
 '?': 47}

In [16]:
#returns a tensor for training
def get_train(book, sentence_length=50):
    sent = []
    next_c = []
    #get varying sentences of similar sizes - step for a size of 5 - 5 is an arbitraily chosen number
    for i in range(0, len(book)-sentence_length, 5):
        begin_char = i
        end_char = i+sentence_length
        #combine sentence to a sentence list
        sent.append(book[begin_char: end_char])
        #get the next character after the sentence
        next_c.append(book[end_char])
        
    print(sent[:5])
    print(next_c[:5])
    
    #turn into Tensors for training the model - using one hot encoding method
    # 1 if char in sentence and 0 otherwise
    unique_c_length = len(set(book)) # number of unique characters in text
    num_sent = (len(sent)) #numer of sentences created
    x = np.zeros((num_sent, sentence_length, unique_c_length), dtype=np.int8)
    y = np.zeros((num_sent, unique_c_length), dtype=np.int8)

    for i, sentence in enumerate(sent):
        for k, c in enumerate(sentence):
            c_ind = char_to_int[c]
            x[i, k, c_ind] = 1
        
        next_c_ind = char_to_int[next_c[i]]
        y[i, next_c_ind] = 1
    
    
    return x,y

["project gutenberg's the adventures of sherlock hol", "ct gutenberg's the adventures of sherlock holmes b", "tenberg's the adventures of sherlock holmes by art", "rg's the adventures of sherlock holmes by arthur c", 'the adventures of sherlock holmes by arthur conan ']
['m', 'y', 'h', 'o', 'd']


In [25]:
sentence_length = 50
num_unique_chars = len(set(sub_books_combined))
print("sentence_length: ", sentence_length)
print("num unique_chars: ", num_unique_chars)

#create tensors
x, y = get_train(sub_books_combined, sentence_length=sentence_length)
print(x.shape)
print(y.shape)



sentence_length:  50
num unique_chars:  48


In [17]:
x.shape

(999990, 50, 48)

In [18]:
y.shape

(999990, 48)

In [28]:
#save models that perform best
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

#create network
model = Sequential()
#128 nodes are used b/c the current network is small
model.add(LSTM(128, input_shape=(sentence_length, num_unique_chars)))
model.add(Dropout(0.2))
#add the output layer
model.add(Dense(num_unique_chars, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x, y, epochs=20, batch_size=128, callbacks=[checkpoint])

Epoch 1/20
116480/999990 [==>...........................] - ETA: 16:35 - loss: 2.6235

KeyboardInterrupt: 

### save architecture to json
### save final weights to h5 file

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_1_architecture.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_1.h5")
print("Saved model to disk")

In [ ]:
#creates new sentence
def create_passage(model):
    #complete passage string
    gen_sentence = ''

    #create sentence
    start_ind = random.randint(0, len(sub_books_combined) - sentence_length - 1)

    sentence = sub_books_combined[start_ind: start_ind+sentence_length]
    gen_sentence += sentence
    print('original sentenc: ', sentence)

    for i in range(500):
    #turn sentence into model format
    x_pred = np.zeros((1, sentence_length, len(char_to_int)))
    for k, c in enumerate(sentence):
        x_ind = char_to_int[c]
        x_pred[0, k, x_ind] = 1

    #predict next character - returns predicted probabilities
    prob_c = model.predict(x_pred, verbose=0)[0]
    #turn to float64 - mulitnomial gives error otherwise
    prob_c = np.asarray(prob_c).astype('float64')
    #     print(prob_c)
    #sample from the probability 
    log_prob = np.log(prob_c) / 0.5
    #     print(log_prob)
    exp_prob = np.exp(log_prob)
    #     print(exp_prob)
    pred_prob = exp_prob/np.sum(exp_prob)
    #     print(pred_prob)
    p = np.random.multinomial(1, pred_prob, 1) 
    #     print(p)
    prob_ind = np.argmax(p)
    #     print(prob_ind)




    #turn int to character
    next_c = int_to_char[prob_ind]
    #add character to generated sentence
    gen_sentence += next_c
    #     print(gen_sentence)
    #get new sentence by sliding to index of sentence
    sentence = sentence[1:]+next_c


    return gen_sentence

      
    
create_passage(model)

When I want to run the model and predict later

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
# evaluate loaded model on test data/ predict
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))